In [1]:
import requests

requests.packages.urllib3.disable_warnings()

import ssl

 

try:

    _create_unverified_https_context = ssl._create_unverified_context

except AttributeError:

    # Legacy Python that doesn't verify HTTPS certificates by default

    pass

else:

    # Handle target environment that doesn't support HTTPS verification

    ssl._create_default_https_context = _create_unverified_https_context


In [2]:
from tensorflow import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Layer
from tensorflow.keras import backend as K
import numpy as np

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [4]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

In [5]:
x_train = np.pad(x_train, ((0,0),(2,2),(2,2),(0,0)), 'constant') #2,3번쨰 축 2픽셀씩 0으로 패딩
x_test = np.pad(x_test, ((0,0),(2,2),(2,2),(0,0)), 'constant')
x_train = x_train / 255
x_test = x_test / 255

In [6]:
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10) # 정수 형태 레이블 데이터를 one-hot encoding

In [7]:
class RBFLayer(Layer):
    def __init__(self, units, gamma, **kwargs) :
        super(RBFLayer, self).__init__(**kwargs)
        self.units = units #RBF 레어어의 출력 크기
        self.gamma = K.cast_to_floatx(gamma) #RBF 함수의 매개변수 gamma(random값)

    def build(self, input_shape):
        self.mu = self.add_weight(name='mu',
                                  shape=(int(input_shape[1]), self.units),
                                  initializer='uniform',
                                  trainable=True)
                                #RBF 함ㅜ의 중심(mu)를 위한 가중치
    def call(self, inputs):
        diff = K.expand_dims(inputs) - self.mu
        l2 = K.sum(K.pow(diff,2), axis=1)
        res = K.exp(-1*self.gamma*l2)
        return res
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.units)
        

In [8]:
model = Sequential()
model.add(Conv2D(6, kernel_size=(5,5), activation='tanh',
                 input_shape=(32,32,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(16, kernel_size=(5,5), activation='tanh'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(120, activation='tanh'))
model.add(Dense(84, activation='tanh'))
model.add(RBFLayer(10, 0.5))

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(), metrics=['accuracy'])
model.fit(x_train, y_train, epochs=5, verbose=1,
          validation_data=(x_test,y_test))

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.8874 - loss: 0.4093 - val_accuracy: 0.9835 - val_loss: 0.0547
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9834 - loss: 0.0536 - val_accuracy: 0.9843 - val_loss: 0.0477
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9893 - loss: 0.0350 - val_accuracy: 0.9875 - val_loss: 0.0404
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9929 - loss: 0.0233 - val_accuracy: 0.9889 - val_loss: 0.0395
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9942 - loss: 0.0183 - val_accuracy: 0.9869 - val_loss: 0.0431


In [10]:
score = model.evaluate(x_test, y_test)
print('테스트 정확도 :', score[1])

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9824 - loss: 0.0558
테스트 정확도 : 0.9868999719619751
